# Финальный проект по А/B-тестированию

<div  style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px ">
    Цель проекта

Провести оценку результатов A/B-теста на основании датасетов с действиями пользователей.
Оценить корректность проведения теста и проанализировать результаты.
Необходимо удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно, проверить равномерность распределения пользователей по тестовым группам и правильность их формирования.
</div>

<div  style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px ">
    Техническое задание

Название теста: recommender_system_test;  
Группы: А (контрольная), B (новая платёжная воронка);  
Дата запуска: 2020-12-07;  
Дата остановки набора новых пользователей: 2020-12-21;
Дата остановки: 2021-01-04;  
Ожидаемое количество участников теста: 15% новых пользователей из региона EU;  
Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;  
Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов:
* конверсии в просмотр карточек товаров — событие product_page
* просмотры корзины — product_cart
* покупки — purchase.
Проверить корректность проведения теста и проанализировать полученные результаты.
</div>

<div  style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px ">
    Описание данных:

Календарь маркетинговых событий на 2020 год (ab_project_marketing_events)
* name — название маркетингового события;
* regions — регионы, в которых будет проводиться рекламная кампания;
* start_dt — дата начала кампании;
* finish_dt — дата завершения кампании.

Пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года (final_ab_new_users)
* user_id — идентификатор пользователя;
* first_date — дата регистрации;
* region — регион пользователя;
* device — устройство, с которого происходила регистрация.

События новых пользователей в период с 7 декабря 2020 по 4 января 2021 года (final_ab_events)
* user_id — идентификатор пользователя;
* event_dt — дата и время события;
* event_name — тип события;
* details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

Таблица участников тестов (final_ab_participants)
* user_id — идентификатор пользователя;
* ab_test — название теста;
* group — группа пользователя.
</div>

<div  style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px ">
    План работы
    
1. Предобработка данных.  
2. Оцените корректность проведения теста:  
    2.1. Период набора пользователей в тест  
    2.2. Регион регистрации пользователей  
    2.3. Равномерность распределения пользователей по группам теста и динамика набора пользоваталей в группы  
    2.4. Пересечений с конкурирующим тестом  и поиск пользователей, участвующих в двух группах теста одновременно  
    2.5. Даты совершения событий участниками теста и соответствие техническому заданию  
    2.6. Активность пользователей: все ли зарегистрированные пользователи прошли авторизацию и совершали переход по продуктовой воронке  
    2.7. Горизонт анализа: рассчитайте лайфтайм совершения события 14 дней
3. Проведите исследовательский анализ данных:  
    3.1. Распределение количества событий на пользователя в разрезе групп теста  
    3.2. Динамика количества событий в группах теста по дням   
    3.3. Влияние маркетинговых активностей на тест  
    3.4. Продуктовые воронки для двух групп теста 
4. Проверка статистической разницы долей z-критерием  
5. Общий вывод
</div>

## 1. Предобработка данных

In [67]:
# Загрузка библиотек
import pandas as pd
from scipy import stats as st
import math as mth
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.simplefilter('ignore')

In [ ]:
# Открытие датасетов
try:
    marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
    new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
    events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
    participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')
except:
    marketing_events = pd.read_csv('C:/Users/COVIDisLIE/Downloads/ab_project_marketing_events.csv')
    new_users = pd.read_csv('C:/Users/COVIDisLIE/Downloads/final_ab_new_users.csv')
    events = pd.read_csv('C:/Users/COVIDisLIE/Downloads/final_ab_events.csv')
    participants = pd.read_csv('C:/Users/COVIDisLIE/Downloads/final_ab_participants.csv')

In [ ]:
display(marketing_events.head(5))
display(marketing_events.shape)

In [ ]:
display(new_users.head(5))
display(new_users.shape)

In [ ]:
display(events.head(5))
display(events.shape)

In [ ]:
display(participants.head(5))
display(participants.shape)

In [ ]:
# Выводим общюю информацию
print(marketing_events.info(), new_users.info(), events.info(), participants.info())

In [ ]:
# Преобразование формата времени
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'])

In [ ]:
# Проверим данные на пропуски 
print("Пропуски данных:", marketing_events.isna().sum(), 
     new_users.isna().sum(),
     events.isna().sum(),
     participants.isna().sum())

In [ ]:
# Проверим данные на полные дубликаты
print("Полные дубликаты данных:", marketing_events.duplicated().sum(), 
      new_users.duplicated().sum(), 
      events.duplicated().sum(), 
      participants.duplicated().sum())

In [ ]:
# Узнаем какие маркетингового события проводились, какие регионы учувствовали
print("Наименование маркетингового события:", marketing_events['name'].unique())
print("Регион:", marketing_events['regions'].unique() )

In [ ]:
# Узнаем Наименование девайсов, какие регионы учувствовали
print("Наименование девайсов:", new_users['device'].unique())
print("Регион:", new_users['region'].unique())
print("Количество пользователей", new_users['user_id'].nunique())

In [ ]:
# Узнаем сколько дней велась регистрация 
print(min(new_users['first_date']))
print(max(new_users['first_date']))

Увидели, что наименование Датасета "Пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года" не соответсвует его содержимому. Регистрация велась 22 декабря 2020 года, о чем не указано в ТЗ.

In [ ]:
# Узнаем Наименованиетипа событий
print("Наименование типа событий:", events['event_name'].unique())
print("Количество пользователей", events['user_id'].nunique())

In [ ]:
# Узнаем сколько дней велась регистрация 
print(min(events['event_dt']))
print(max(events['event_dt']))

Увидели, что наименование Датасета "События новых пользователей в период с 7 декабря 2020 по 4 января 2021 года" не соответсвует его содержимому. В датасете отражены события с 7 по 30 декабря 2020 года.

In [ ]:
# Узнаем Наименованиетипа теста и количество пользователей
print("Наименование теста:", participants['ab_test'].unique())
print("Количество пользователей", participants['user_id'].nunique())
print("Количество пользователей по группам", participants['group'].value_counts())

## 1. Вывод
1. В датасете "Календарь маркетинговых событий на 2020 год" данные полные и не имеют дубликатов. Всего было проведено 14 маркетинговых событий в 6 группах регионов. Данные столбцов start_dt и  finish_dt  преобразованы формата времени datetime.  
2. В датасете "Пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 " данные полные, и не имеют дубликатов. Всего зарегистрировано 61733 пользователя из 4 регионов. Пользователи использовали для регистрации 4 вида девайсов. Регистрация велась с 7 по 22 декабря 2020 года. В ТЗ было заявлено о регистрации с 7 по 21 декабря 2020 года.  
3. В датасете "События новых пользователей в период с 7 декабря 2020 по 4 января 2021 года" данные не имеют дубликатов. В данных представлена информация о 58703 пользователях которые совершали 4 типа событий в период с 7 по 22 декабря 2020 года. В столбце details  377577, данные пропуски не требуют обработки так ка являются дополнительным комментарием о данных.  
4. В датасете "таблица участников тестов" данные полные и не имеют дубликатов. В данных представлена информация о 16666 пользователях которые учувствуют в тесте интерфейса и тесте системы. В группе А 9655 пользователей, в группе В 8613 пользователя.

Так же проведено преобразование формата времени в тип данных datetime.

## 2. Оценка корректности проведения теста

### 2.1. Период набора пользователей в тест

Ранее мы выяснили, что регистрация пользователей проходила до 23 декабря 2020 года.
Выясним сколько пользователей зарегистрировались, во временной промежуток не входящий в ТЗ.
И сколько пользователей подходят под описание ТЗ и должны войти в тест.

In [ ]:
# Сколько пользоваталей не по ТЗ
users_not_tz = new_users.query('first_date == "2020-12-22" or first_date == "2020-12-23"')['user_id']
print('Пользователеи зарегестрированные после 2020-12-21:', users_not_tz.nunique())

In [ ]:
# Сколько пользоваталей по ТЗ
users_tz = new_users.query('first_date >= "2020-12-07" and first_date <= "2020-12-21"')['user_id']                                                                     
print('Пользователи зарегестрированные по ТЗ:', users_tz.nunique())

In [ ]:
# Ищем пересечение этих групп пользователей
pd.Series(list(set(users_tz).intersection(set(users_not_tz))))

Для того, что бы тест был проведен по ТЗ пользователей, которые зарегистрировались после 21 декабря 2020, нажно убрать из таблиц new_users, events и participants.

In [ ]:
# Удаляем пользоваталей не соответсвующих ТЗ
new_users = new_users.query('first_date >= "2020-12-07" and first_date <= "2020-12-21"')
events = events.query('user_id not in @users_not_tz')
participants = participants.query('user_id not in @users_not_tz')

В соответсвии с ТЗ необходимо избавиться от 'interface_eu_test' в таблице participants

In [ ]:
# Удаляем 'interface_eu_test' в таблице participants
participants_test = participants.loc[participants['ab_test'] == 'recommender_system_test']


### 2.2. Регион регистрации пользователей

В соответсвии с ТЗ исследуем только пользователей из EU региона и уточним сколько таких пользоваталей

In [ ]:
# Удаляем 'N.America' 'APAC' 'CIS' в таблице new_users
new_users_eu = new_users.loc[new_users['region'] == 'EU']
display(new_users_eu.shape) 

In [ ]:
# Объеденим participants и events по 'user_id'
ab_test = participants_test.merge(events, on = 'user_id', how='left')

In [ ]:
# Избавимся от пользоваталей которые не совершали  действия ('event_name')
#ab_test = ab_test.dropna(subset=['event_name'])

In [ ]:
# Объеденим ab_test и new_users по 'user_id'
ab_test = ab_test.merge(new_users_eu, on = 'user_id', how='left')

In [ ]:
# Избавимся от пользоваталей которые не имеют региона ('region')
ab_test = ab_test.dropna(subset=['region'])
ab_test['first_date'] = pd.to_datetime(ab_test['first_date']). dt.date
ab_test['event_dt'] = pd.to_datetime(ab_test['event_dt']). dt.date
ab_test.sample(5)

In [ ]:
# Ищем % новых пользователей из региона EU

participants_test = participants_test.merge(new_users, on='user_id', how = 'inner')

print('Доля новых пользователей из региона EU составляет:',\
round(participants_test.query('region == "EU"')['user_id']\
.nunique() / new_users.query('region == "EU"')['user_id'].nunique()*100),'%'
)

### 2.3. Равномерность распределения пользователей по группам теста и динамика набора пользоваталей в группы

In [ ]:
# Ищем количество пользоваталей по группам
print("Количество пользователей по группам", ab_test.groupby('group')['user_id'].nunique())

In [ ]:
# Визуализация динамики набора пользователей по группам
plot=ab_test.groupby(['first_date', 'group'])['user_id'].count().reset_index()
plot.index = plot['first_date']

plt.figure(figsize = (15, 10))
plt.grid(True)
plt.title('Динамика набора пользователей по группам', size=12)
sns.barplot(data = plot, x = plot['first_date'], y = 'user_id', hue = 'group')
plt.ylabel('Количество пользоваталей')
plt.xlabel('Дата')
plt.xticks(rotation = 45)
plt.show()

Выясним сколько пользователей зарегистрировались, во временной промежуток не входящий в ТЗ.
И сколько пользователей п
На графике "Динамика набора пользователей по группам" мы видим превышение количество пользователей группы А к группе В в разы.
По данным изученным выше мы знаем, что в три раза превышает размер группы А к размеру группы В. 
Имеется два пика набора пользователей в группу В 07 и 16 декабря.
Набор пользователей в группу А резко возрос на второй неделе периода регистрации.

### 2.4. Пересечений с конкурирующим тестом  и поиск пользователей, участвующих в двух группах теста одновременно

In [ ]:
# Ищем пересечения с конкурирующим тестом 
users_two_test = participants.groupby('user_id').agg({'ab_test': ['nunique', 'unique']})
users_two_test.columns = ['num_groups', 'group_names']
users_two_test = users_two_test.query('num_groups > 1').reset_index()
display(users_two_test.sample(2))
users_two_test.count()

In [ ]:
# Избавимся от пользоваталей из группы В участвующей в interface_eu_test
index_names = participants[ (participants['group'] ==  "B") & (participants['ab_test'] == "interface_eu_test")].index
ab_test.drop(index_names, inplace = True) 

In [ ]:
# Ищем пользоваталей участвующих в двух группах теста одновременно
not_unique_users = participants.groupby(['user_id', 'group']).size().reset_index()
not_unique_users.columns = ['user_id', 'group', 'count']
not_unique_users = not_unique_users.query('count > 1').sort_values(by='user_id')
print(not_unique_users.shape)
print(not_unique_users.sample(2))

В данных нет пользователей участвующих в двух группах теста одновременно. Пользователи участвующих тесте согласно ТЗ и пользователей из группы А участвующих в конкурирующем тесте одновременно 482 пользователя.

По ТЗ мы знаем, что группа А это контрольная группа в тестах, то есть функциональность не изменена, можно предположить, что контрольная группа теста interface_eu_test может быть использована в нашем тесте условно "безопасно" и можно оставить этих пользователей.

### 2.5. Даты совершения событий участниками теста и соответствие техническому заданию

In [ ]:
# Разделим на группы для визуализации
group_A = ab_test.query('group == "A"')
group_B = ab_test.query('group == "B"')

In [ ]:
# Строим графики каличества событий по дням
plt.figure(figsize=(7, 14))

ax = group_A.pivot_table(index='event_dt', values='user_id', columns='event_name', aggfunc='count').plot.bar(stacked=True, ylim=(0, 2000))
ax.set_title("Группа A")
plt.xlabel('Дата')
plt.ylabel('Количество событий');

ax = group_B.pivot_table(index='event_dt', values='user_id', columns='event_name', aggfunc='count').plot.bar(stacked=True, ylim=(0, 2000))
ax.set_title("Группа B")
plt.xlabel('Дата')
plt.ylabel('Количество событий');

plt.show()

В группе А видим пик около 21 декабря. Так же видим резкий рост и относительную стабилизацию 14 декабря. 
В группе В наблюдаем снижение активности и последующее ее затухание после 21 декабря.
Графики так же явно отражают, насколько группа В малочисленнее группы А.
В ТЗ заявлено окончание теста 4 января на самом деле данные заканчиваются уже 30 декабря.

В связи с тем, что сбор данных закончился раньше определенногов ТЗ срока, мы можем предположить что, либо имеет место сбой при сборе данных, либо тест по какой - то причине закончили раньше.  
Явно имеем технические проблемы при распределении пользователей по группам в связи с перекосом численности и вероятно при сборе информации. 

### 2.6. Активность пользователей: все ли зарегистрированные пользователи прошли авторизацию и совершали переход по продуктовой воронке

In [ ]:
# считаем активность пользователей групп А и В
ab_test['day'] = pd.to_datetime(ab_test['event_dt']). dt.date

# Рассмотрим среднее количество событий в день на пользователя
events_by_day = (ab_test.groupby(['group','day']).agg({'event_dt':'count', 'user_id':'nunique'}).reset_index())
events_by_day.columns = ['group','day','events','n_users']
events_by_day['events_per_users'] = round(events_by_day['events'] / events_by_day['n_users'],2)

In [ ]:
# проверим удален ли из датасета тест 'interface_eu_test' согласно ТЗ
ab_test['ab_test'].unique()

In [ ]:
# Выведем количество пользоваталей без действий
print("Количество пользователей, которые не совершали действий поле регистрации:",ab_test['event_name'].isna().sum())

In [ ]:
# Выведем количество пользоваталей без действий по группам теста 
print("Количество пользователей, которые не совершали действий поле регистрации в группе А:",  ab_test.query('group == "A"')['event_name'].isna().sum()) 
print("Количество пользователей, которые не совершали действий поле регистрации в группе В:", ab_test.query('group == "B"')['event_name'].isna().sum())

In [ ]:
# Разделим на группы для визуализации
events_by_day_users_a = events_by_day.query('group == "A"')
events_by_day_users_b = events_by_day.query('group == "B"')

In [ ]:
# Строим графи Распределение среднего количества событий по дням
fig = go.Figure(data=[
    go.Bar(name='Cреднее количество событий в группе А', x=events_by_day_users_a['events_per_users'], y=events_by_day_users_a['day'], orientation='h'),
    go.Bar(name='Cреднее количество событий в группе B', x=events_by_day_users_b['events_per_users'], y=events_by_day_users_b['day'], orientation='h')
])
fig.update_layout(barmode='stack', title='Распределение среднего количества событий по дням', xaxis_title='Среднее количество событий', yaxis_title='Дата')
fig.show()

Избавившись от разницы между размерами групп сравниваем их не по абсолютному значению, а по  распределению среднего количества событий по дням на пользователей.
В среднем количество событий на пользователя одинаково в обеих группах. Меньше всего событий совершали 11 - 13 декабря.

In [ ]:
# Количество разных событий
events['event_name'].value_counts()

Действия пользователя: login (вход) -> product_page(страница товара) -> product_cart (корзина) -> purchase (покупка). Странно, что покупок больше, чем просмотров корзины вероятно, в приложении есть быстрая покупка и корзина не нужна.
Уже на этом этапе мы видим, что существуют пользоватали, которые не совершали действий после авторизации. И таких сабытий было около 60 тысяч.

In [ ]:
# Строим воронку событий
funnel = events.groupby('event_name').agg({'user_id': 'nunique'}).reset_index()
funnel = funnel.reindex([0,2,1,3])


fig = go.Figure(go.Funnel(y = funnel['event_name'], x = funnel['user_id'], textinfo = "value+percent initial+percent previous"))
fig.update_layout(title='Воронка событий пользователей', xaxis_title='Количество событий')
fig.show()

Конверсия в просмотр страницы составляет (product_page) 66%.
Просматривают корзину (product_cart) - 33% от всех пользователей или 49% от тех, кто посмотрел страницу продукта.
Конверсия в целевое действие (purchase) составляет 32%.

Мы выяснили, что существует 2870 пользователей которые после регистрации, не совершали никаких действий. Из них в группе А: 1030 пользователей, в группе В: 1840 пользователей.  
Что бы провести дальнейший тест данные пользователи нам нужны для вывода о том, что не все пользователи авторизуются и совершают действия, но для построения воронки они нам не нужны, так как их конверсия в дальнейшие действия не произошла.

### 2.7. Горизонт анализа: рассчитайте лайфтайм совершения события 14 дней

In [ ]:
# Избавимся от событий с лайфтаймом больше, чем горизонт событий (14 дней)
ab_test['delta'] = ab_test['event_dt'] - ab_test['first_date']

horizon_14 = pd.Timedelta(days=14)
ab_test1 = ab_test[ab_test['delta'] <= horizon_14]
ab_test1.sample(5)

По ТЗ ожидаемый эффект должен быть достигнут за 14 дней с момента регистрации в системе пользователи

In [ ]:
# проверим сколько пользоваталей в датасете совершали события за преедлами лайфтайма
after_garizont=ab_test[ab_test['delta'] > horizon_14]
display(after_garizont.shape)

Получили, что 592 пользователя вышли за горизонт событий и не соответствуют ТЗ.

In [ ]:
# первое и последнее действие + delta событий
events_min_max = ab_test1.groupby('user_id').agg({'event_dt':['min', 'max']}).reset_index()
events_min_max['time_event']=ab_test['event_dt'] - ab_test['first_date']
events_min_max['time_event1']=events_min_max['time_event'].dt.days
events_min_max.sample(6)


In [ ]:
plt.figure(figsize=(15, 6))
ax = sns.histplot(data=events_min_max, x='time_event1',  kde=True, bins=25)
plt.title('Гистограмма распределения количества дней до достижения события')
plt.xlabel('Количество дней')
plt.ylabel('Частота')
plt.show()

Проведем расчеты и узнаем сколько времени в среднем в днях потребовалось пользователям для достижения каждого из видов действий

In [ ]:
event_login=ab_test1.query('event_name == "login"')['delta'].mean()
event_product_page=ab_test1.query('event_name == "product_page"')['delta'].mean()
event_product_cart=ab_test1.query('event_name == "product_cart"')['delta'].mean()
event_purchase=ab_test1.query('event_name == "purchase"')['delta'].mean()
delta=ab_test1['delta'].mean()
event_login

In [ ]:
print("Количество дней за которое пользователь в среднем доходил до авторизации:",event_login)
print("Количество дней за которое пользователь в среднем доходил до просмотр страницы:",event_product_page)
print("Количество дней за которое пользователь в среднем доходил до корзины:",event_product_cart)
print("Количество дней за которое пользователь в среднем доходил до покупки:",event_purchase)
print("Количество дней с среднем до соврешения любогшо из дейсвий:",event_purchase)

Мы можем увидеть, что в среднем с момента регистрации и до каких- либо действий от пользователя проходило около трех дней.
При более подробном рассмотрении мы понимаем, что при движении пользователя по воронке время до действий сокращалось, это может говорить о том, что на каждом этапе воронки оставались более целеустремленные пользователи.

## 2. Вывод
Данные не соответствуют условиям теста и в нем недостаточное количество участников. Число новых пользователей, участвовавших в событиях, меньше ожидаемого - 3481 вместо 6000.  Имеется 2870 пользоваталей которые ничего не делали после регистрации. Из них 1030 пользователей находятся в группе А, а 1840 - в группе В. 
Были обнаружены пользователи(482), принявшие участие одновременно в двух тестах. Так как группа А это контрольная группа в тестах, то есть функциональность не изменена, можно оставить этих пользователей.  
Распределение пользователей по группам непропорциональное, контрольная группа почти в 3 раза больше тестовой.  
В связи с тем, что сбор данных закончился раньше определенного в ТЗ срока, мы можем предположить что, либо имеет место сбой при сборе данных, либо тест по какой - то причине закончили раньше.  
Доля новых пользователей из Европы, попавших в тест - удовлетворяет техническому заданию и составляет 15%.   
События в собранных данных обрываются раньше установленного времени.  
Так же можно заметить нюанс, что имеются пользователи, которые совершили платёж не заходя в корзину. Вероятнее всего, они купили только 1 товар. Это может говорить о том, что на платформе нестрогая воронка продаж и можно приобрести продукт минуя некоторые этапы.  
Мы можем увидеть, что в среднем с момента регистрации и до каких- либо действий от пользователя проходило около трех дней.
Исходя из базового показателя конверсии в 50% данный уровень не достигается.
Таким образом, не все условия, указанные в ТЗ были выполнены. Корректность проведения А/В теста не соблюдается.

## 3. Исследовательский анализ данных

### 3.1. Распределение количества событий на пользователя в разрезе групп теста

Распределение количества событий на пользователя в разрезе групп теста: постройте гистограмму распределения этой величины в разрезе групп и сравните её средние значения между собой у групп теста;

In [ ]:
#количество событий по пользователями и событиям  
users_events = (ab_test.groupby(['user_id'], as_index=False)['event_name'].count()
                              .rename(columns={'event_name':'events_count'})
                              .merge(ab_test[['user_id', 'group']], on='user_id')
                              .drop_duplicates())

users_events.sample(5)

In [ ]:
print ('Усредненное количество событий на пользователя ', \
    round (users_events['events_count'].mean() ))

In [ ]:
print ('Усредненное количество событий на пользователя в группе А = ', \
    round (users_events.query('group == "A"')['events_count'].mean()) )

In [ ]:
print ('Усредненное количество событий на пользователя в группе B = ', \
    round (users_events.query('group == "B"')['events_count'].mean()) )

In [ ]:
# избавимся от пользоватлеей без событий длля построения гистаграммы
users_events=users_events[users_events['events_count']!=0]


In [ ]:
# Строим Гистограмму распределения количества событий по пользователям и событиям в группах
plt.figure(figsize=(15, 6))
ax = sns.histplot(data=users_events, x='events_count', hue='group', kde=True, bins=28)
plt.title('Гистограмма распределения количества событий по пользователям и событиям в группах')
plt.xlabel('Количество событий')
plt.ylabel('Частота')
plt.legend(labels=users_events['group'].unique(), title="Группы", fontsize="x-large", title_fontsize="large")
plt.show()

По графику видно, что динамика распределения событий в группах А и В совпадает, но за счет численности групп имеет разное количество событий.
Среднее количество событий на пользователя - 7 событий. А именно по группе А - 7 событий, по группе В- 6 событий.
Что так же говорит, о том, что группы примерно равны друг другу по поведению пользователей.

### 3.2. Динамика количества событий в группах теста по дням

In [ ]:
# создаем DataFrame event_by_date, на основнаии которого будем строить гарфик
event_by_date = ab_test.groupby(['event_dt', 'group'])['user_id'].count().reset_index()

In [ ]:
# строим график
plt.figure(figsize=(15, 4))
sns.set_style("whitegrid")
ax = sns.lineplot(data=event_by_date, x='event_dt', y='user_id', hue='group')
ax.set_xticks(event_by_date['event_dt'])
ax.set_title('Распределение количества событий в группах по дням')
plt.xlabel("Дата события")
plt.ylabel("Количество событий")
plt.xticks(rotation=45)
plt.show();

По графику мы видим, что наблюдается резкий рост числа событий в группе А в период с 13 по 21 декабря;
Группа В ведет себя походим образом, но за счет ее численности количество событий пропорционально меньше.

### 3.3. Влияние маркетинговых активностей на тест 

In [ ]:
# удаляем все события и регионы которые не соответсвуют ТЗ
marketing_events1 = marketing_events[marketing_events['regions'].str.contains("EU")]
marketing_events1 = marketing_events1.query('start_dt >= "2020-12-07" and start_dt <= "2021-01-03"')
marketing_events1

In [ ]:
# Избавимся от пользоваталей которые не совершали  действия ('event_name')
ab_test_events = ab_test.dropna(subset=['event_name'])

plt.figure(figsize=(15, 7))
sns.histplot(x='event_dt', data=ab_test_events, palette='crest')
plt.title('Распределение событий по времени')
plt.xlabel('Дата')
plt.ylabel('Количество событий')
plt.show()

Мы проверили какие маркетинговые события проходили во время проведения теста и узнали, что с 25 декабря 2020 началась акция Christmas&New Year Promo Построили визуализацию распределения событий по датам и увидели, что данная акция не повлияла на количество событий, так как снижения количества событий началось до начала акции.

### 3.4. Продуктовые воронки для двух групп теста 

In [ ]:
# определяем переменные для создания воронки
funnel_A = ab_test[ab_test['group']=="A"].groupby('event_name').agg({'event_name':'count', 'user_id':'nunique'})
funnel_A.columns = ['event_count','user_count']
funnel_A = funnel_A.sort_values(by = 'event_count', ascending = False).reset_index().reindex([0,1,3,2])

funnel_B = ab_test[ab_test['group']=="B"].groupby('event_name').agg({'event_name':'count', 'user_id':'nunique'})
funnel_B.columns = ['event_count','user_count']
funnel_B = funnel_B.sort_values(by = 'event_count', ascending = False).reset_index().reindex([0,1,3,2])

In [ ]:
funnel_A

In [ ]:
funnel_B

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Group A',
    y = funnel_A['event_name'],
    x = funnel_A['user_count'],
    textinfo = "value+percent initial"))
fig.update_layout(
    title = {
        'text' : 'Конверсия событий по группе А',
        'x':0.5
    }
)
fig.show()

Для группы А конверсия в просмотр страницы составляет (product_page) 65%. Просматривают корзину (product_cart) - 30%/ Конверсия в целевое действие (purchase) составляет 32%

In [ ]:
fig = go.Figure()


fig.add_trace(go.Funnel(
    name = 'Group B',
    y = funnel_B['event_name'],
    x = funnel_B['user_count'],
    textinfo = "value+percent initial"))
fig.update_layout(
    title = {
        'text' : 'Конверсия событий по группе В',
        'x':0.5
    }
)

fig.show()



Для группы В конверсия в просмотр страницы составляет (product_page) 56%. Просматривают корзину (product_cart) - 28%/ Конверсия в целевое действие (purchase) составляет 28%

## 3. Вывод 
Динамика распределения событий в группах А и В совпадает, но за счет численности групп имеет разное количество событий.
Среднее количество событий на пользователя - 7 событий. А именно по группе А - 7 событий, по группе В- 6 событий.
Что так же говорит, о том, что группы примерно равны друг другу по поведению пользователей.
Наблюдается резкий рост числа событий в группе А в период с 13 по 21 декабря. Группа В ведет себя походим образом, но за счет ее численности количество событий пропорционально меньше.  
С 25 декабря 2020 началась акция Christmas&New Year Promo, которая  не повлияла на количество событий, так как снижения количества событий началось до начала акции (21.12.2020).  
Для группы А конверсия в просмотр страницы составляет (product_page) 65%. Просматривают корзину (product_cart) - 30%/ Конверсия в целевое действие (purchase) составляет 32%  
Для группы В конверсия в просмотр страницы составляет (product_page) 56%. Просматривают корзину (product_cart) - 28%/ Конверсия в целевое действие (purchase) составляет 28%
Можно заметить, что имеются пользователи, которые совершили платёж не заходя в корзину, что говорит о том на платформе нестрогая воронка продаж и можно приобрести товары через быструю покупку.
А группа (контрольная) показала рост конверсии на всех этапах воронки, однако целевое действие увеличилось только на 2%.

## 4. Проверка статистической разницы долей z-критерием

Проверим гипотезу: Конверсия в действия у групп пользователей различаються

H_0: Доли не имеют статистических различий

H_1: Доли имеют статистические различия

alpha = 0.05

In [ ]:
#создаем табличку по столбцам 'delta', 'group', 'event_name' и с кол-вом событий
conversion = (ab_test.groupby(['delta', 'group', 'event_name'], as_index=False)['user_id'].count()
                  .rename(columns={'user_id':'events_count'}))
conversion.head(5)

In [ ]:
# функция проверки Z - критерием
def z_test(successes, trials, alpha):
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
    
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = abs(difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1])))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу")
    else:
        print("Не получилось отвергнуть нулевую гипотезу")

In [ ]:
a_count = ab_test.query('group == "A"')['user_id'].nunique()
b_count = ab_test.query('group == "B"')['user_id'].nunique()

product_cart_a = conversion.query('event_name == "product_cart" & group == "A"')['events_count'].values[0]
product_cart_b = conversion.query('event_name == "product_cart" & group == "B"')['events_count'].values[0]

product_page_a = conversion.query('event_name == "product_page" & group == "A"')['events_count'].values[0]
product_page_b = conversion.query('event_name == "product_page" & group == "B"')['events_count'].values[0]

purchase_a = conversion.query('event_name == "purchase" & group == "A"')['events_count'].values[0]
purchase_b = conversion.query('event_name == "purchase" & group == "B"')['events_count'].values[0]

alpha = 0.05
trials = [a_count, b_count]

s = {'product_cart':[product_cart_a,product_cart_b], 'product_page':[product_page_a,product_page_b], 'purchase':[purchase_a,purchase_b]}
for ev in ['product_cart', 'product_page', 'purchase']:
    successes = s[ev]
    print(f'Тестируем событие "{ev}" / successes = {s[ev]}')
    z_test(successes, trials, alpha)
    print()

## 4. Вывод
По метрике - product_cart  доли не имеют статистических различий.  
По метрике - product_page  доли  имеют статистических различий.  
По метрике - purchase  доли  имеют статистических различий.  
В связи с тем, что целевое действие purchase  то мы можем сказать, что ожидаемый эффект в изменении конверсии был достигнут.

## 5. Общий вывод



В датасете "Календарь маркетинговых событий на 2020 год" содержатся полные данные без дубликатов, охватывающие 14 маркетинговых событий, разделенных на 6 групп регионов. Также столбцы start_dt и finish_dt преобразованы в формат времени datetime.  
В датасете "Пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020" имеются полные данные без дубликатов о 61733 зарегистрированных пользователях из 4 регионов. Пользователи использовали 4 различных типа устройств для регистрации. Регистрация проходила с 7 по 22 декабря 2020, что не соответствует указанному периоду с 7 по 21 декабря 2020 в ТЗ.  В датасете "События новых пользователей в период с 7 декабря 2020 по 4 января 2021" отсутствуют дубликаты. Данные содержат информацию о 58703 пользователях, которые выполнили 4 типа событий в период с 7 по 22 декабря 2020. Столбец details содержит 377577 пропущенных значений, которые не требуют обработки, так как являются дополнительным комментарием к данным.
В датасете "таблица участников тестов" имеются полные данные без дубликатов о 16666 пользователях, участвующих в тесте интерфейса и тесте системы. В группе А находится 9655 пользователей, а в группе В - 8613 пользователей.  
Данные не соответствуют условиям теста и имеют недостаточное количество участников - всего 3481 новых пользователей приняли участие вместо запланированных 6000. Было обнаружено около 2870 пользователей, которые после регистрации больше никак не взаимодействовали с нашим продуктом.  Из них 1030 пользователей находятся в группе А, а 1840 - в группе В.  
Было обнаружено 482 пользователя, которые участвовали одновременно в двух тестах. Но так как группа А является контрольной группой и не изменяет функциональность, этих пользователей можно оставить.  
Распределение пользователей по группам неравномерное, контрольная группа превышает размер тестовой группы почти в 3 раза.  
В связи с завершением сбора данных раньше указанного срока в ТЗ, можно предположить наличие ошибки при сборе данных или причины, приведшие к преждевременному завершению теста.  
Доля новых пользователей из Европы, участвующих в тестировании, соответствует требованиям ТЗ и составляет 15%.  
События в собранных данных прекращаются раньше заданного времени.  
Также отмечается наличие пользователей, которые совершили платеж без посещения корзины. Вероятно, они приобрели только один товар, что может указывать на нестрогую воронку продаж и возможность приобретения продукта, минуя некоторые этапы.  
В среднем, после регистрации пользователя проходит около трех дней до того, как он совершает какие-либо действия. Однако этот уровень не достигается базовым показателем конверсии в 50%, что означает, что не все условия, указанные в ТЗ, были выполнены. Проведение А/В теста не было корректным.  
Динамика распределения событий в группах А и В одинакова, но из-за различного числа участников в группах, количество событий различается. В среднем, каждый пользователь совершает 7 событий. В группе А этот показатель также составляет 7 событий, в то время как в группе В - 6 событий. Это говорит о том, что поведение пользователей в двух группах примерно одинаковое.  
В группе А наблюдается резкий рост числа событий в период с 13 по 21 декабря. Группа В также проявляет схожее поведение, но из-за различной численности количество событий пропорционально меньше.  
Начиная с 25 декабря 2020 года, стартовала акция Christmas&New Year Promo, которая не повлияла на количество событий, так как снижение числа событий началось до ее начала (21.12.2020).  
В группе А конверсия в просмотр страницы составляет 65%, просмотр корзины - 30%, а конверсия в целевое действие (покупку) - 32%. В группе В конверсия в просмотр страницы составляет 56%, просмотр корзины - 28%, а конверсия в целевое действие - 28%.  
Также можно заметить, что есть пользователи, которые совершают покупки, не заходя в корзину, что указывает на то, что платформа имеет нестрогую воронку продаж и позволяет приобретать товары через быструю покупку. Контрольная группа показала рост конверсии на всех этапах воронки, но увеличение целевого действия составило всего лишь 2%.  
По метрике "просмотр корзины" доли групп А и В не имеют статистических различий. По метрике "просмотр страницы" доли имеют статистические различия. По метрике "покупка" доли также имеют статистические различия. 

Рекомендации:

Нужно подумать о "клиентском пути" на сайте, дополнительно проанализировать действия и время нахождения клиента на сайте. Довольно странно, что авторизовавшись половина клиентов даже не посмотрела товар. Возможно есть технические проблемы.

Новая платёжная воронка прилично снижает конверсии (на 4%), вместо ожидаемого повышения. Необходимо еще раз запустить тест с соблюдением ТЗ и проанализивровать результаты.


